In [1]:
import re, os
from logerror import TestError

root_folder = "/mnt/hgfs/projlogs/SFW/sops1/SOPS/Performance Test - Performance_Test1"
root_folder = "/mnt/hgfs/projlogs/SFW/hq1/Performance Test - Performance_Test1"
# MILL, POND, OPP1, OPP2, FOX
root_folder = "/mnt/hgfs/projlogs/PSU/OPP1"

test_folder = root_folder #+ '/' + location
files = [ f for f in os.listdir(test_folder) if os.path.isfile(os.path.join(test_folder,f)) and f.find('.log.1')>0 ]

# print('for ', location, 'found log files: ', files)
print('found', len(files), 'log files: ', files)

found 2 log files:  ['PONDF1OC_INJPOND_0.log.1', 'PONDF1OC_INJPOND_0.log.1.lck']


In [2]:

if len(files):
    FNAME = files[0]
else:
    FNAME = "LocalInjectorCOBAPPSTS03_0.log.1"
    
print('processing file: ', test_folder + '/' + FNAME)

with open(test_folder + '/' + FNAME,'r') as f:
    alllog = f.read()

errorlog = re.findall(r'(?<=\n).+?\[SEVERE\].+?id \d+?,[\w\s]+?\):.+?(?=\n)', alllog)
print('severe count:', len(errorlog))

processing file:  /mnt/hgfs/projlogs/PSU/POND/PONDF1OC_INJPOND_0.log.1
severe count: 38648


In [3]:
errors = []
vusers = {}
for err in errorlog:
    time = err[:20]
    usr = re.search(r'user.(\d+?),.cycle', err)
    user = usr.group(1)
    user = str(user).rjust(4,'0')
    cyc = re.search(r'cycle.(\d+?),.step', err)
    cycle = cyc.group(1)
    stp = re.search(r'step\s+?(.+?),\s+?id', err)
    step = stp.group(1)
    errtype = re.search(r'status.(.+?)\)', err)
    error_type = errtype.group(1)
    message = err[err.find('):'):]
    new_record = TestError(time, user, cycle, step, error_type, message, test_folder)
    errors.append(new_record)
    if user in vusers.keys():
        vusers[user].append(new_record)
    else:
        vusers[user] = [new_record]

print('errors:', len(errors))


errors: 38648


In [4]:
sorted_keys = sorted(vusers.keys())
print(">> user, i, e.time, e.cycleid, e.errortype, etype, ")
print('erroring users', len(sorted_keys))
for user in sorted_keys:
    print('\nuser ' + user + ' err count:', len(vusers[user]))
    prior_cyc_index = -1
    consecutive_count = 0
    consecutivness = ''
    for i in range(len(vusers[user])-1,-1,-1):
        e = vusers[user][i]
        try:
            etype = e.validation
        except AttributeError:
            try:
                etype = e.dditem
            except AttributeError:
                etype = '!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!'

        # print('pre: cyc', e.cycleid, ' prior', prior_cyc_index, ' consecutive_count', consecutive_count, 'consecutivness', consecutivness )
        # determine consecutivness
        if prior_cyc_index == -1:
            prior_cyc_index = int(e.cycleid)
        else:
            if prior_cyc_index - int(e.cycleid) <= 1 and i != 0:
                consecutive_count += 1
                prior_cyc_index = int(e.cycleid)
            else:
                consecutivness = ';' + str(e.cycleid) + '-' + str(int(e.cycleid) + consecutive_count) + consecutivness
                consecutive_count = 0
                prior_cyc_index = int(e.cycleid)
        cause = e.determine_casuse()
    if len(consecutivness):
        print('~~', consecutivness[1:])

print(end='\n')
print('8'*80)
print("e.time, user, e.cycleid, e.errortype, e.cause, usr err count")
causes = {}



for user in sorted_keys:
    e = vusers[user][0]
    print(e.time, user, e.cycleid, e.errortype.ljust(30), e.cause, len(vusers[user]), sep=' ; ', end=' ; ')
    if e.cause in ['Warning Message', 'Critical Message', 'Question Message', 'dynamic response']:
        print(e.cause_message, sep=' ; ')
    else:
        print()
    if e.cause in causes.keys():
        causes[e.cause] += 1
    else:
        causes[e.cause] = 1

print('count of erroring users:', len(sorted_keys))
for k,v in causes.items():
    print('cause', k, 'has count', v)


>> user, i, e.time, e.cycleid, e.errortype, etype, 
erroring users 20

user 0000 err count: 2
~~ 1-1

user 0001 err count: 2
~~ 5-5

user 0002 err count: 3
~~ 2-3

user 0003 err count: 3
~~ 0-1

user 0004 err count: 30
~~ 2-2;3-4;8-33

user 0005 err count: 2
~~ 2-2

user 0007 err count: 1

user 0009 err count: 44
~~ 18-60

user 0011 err count: 279
~~ 4-281

user 0012 err count: 1

user 0013 err count: 4805
~~ 6-6;7-4809

user 0015 err count: 1

user 0016 err count: 11825
~~ 5-11828

user 0018 err count: 10548
~~ 1-2;8-8;18-10561

user 0019 err count: 10906
~~ 0-10904

user 0020 err count: 16
~~ 0-1;4-5;14-14;17-22;23-24;26-27

user 0021 err count: 170
~~ 3-3;7-7;9-9;11-176

user 0022 err count: 3
~~ 1-1;1-1

user 0024 err count: 6
~~ 29-29;32-33;35-35;55-55

user 0025 err count: 1

88888888888888888888888888888888888888888888888888888888888888888888888888888888
e.time, user, e.cycleid, e.errortype, e.cause
30/01/2018 10:33:21  ; 0000 ; 1 ; Incorrect Response             ; Incorrect Res